In [ ]:
from biolearn.data_library import DataLibrary
import pandas as pd
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE19711

data = DataLibrary().get("GSE19711").load()
data.metadata, data.dnam

#elastic for selection, boost for prediction

In [43]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

X = data.dnam.transpose()
X_df = pd.DataFrame(X)
y = data.metadata['age']
y = pd.DataFrame(y)
cb = pd.merge(X_df, y, left_index=True, right_index=True)
cb_imputed = SimpleImputer(strategy='mean').fit_transform(cb)
cb = pd.DataFrame(cb_imputed, columns=cb.columns, index=cb.index)

X = cb.drop('age', axis=1)
y = cb['age']

cb

,cg00000292,cg00002426,cg00003994,cg00005847,cg00006414,cg00007981,cg00008493,cg00008713,cg00009407,cg00010193,...,cg27654142,cg27655855,cg27655905,cg27657249,cg27657283,cg27661264,cg27662379,cg27662877,cg27665659,age
GSM491937,0.756367,0.797859,0.068605,0.131004,0.076355,0.031687,0.946473,0.030616,0.073854,0.594014,...,0.044225,0.818353,0.088685,0.161284,0.054350,0.253198,0.028654,0.045494,0.046754,68.0
GSM491938,0.834702,0.859538,0.067469,0.197486,0.096817,0.027651,0.956317,0.036103,0.080879,0.568817,...,0.193395,0.851130,0.074006,0.236641,0.060951,0.427664,0.022261,0.044785,0.051954,81.0
GSM491939,0.774165,0.769661,0.056937,0.140949,0.156980,0.027933,0.952377,0.038230,0.064909,0.575405,...,0.075082,0.822084,0.073700,0.187616,0.054692,0.210370,0.027410,0.044007,0.046282,56.0
GSM491940,0.799517,0.854328,0.063802,0.168209,0.086761,0.029277,0.961985,0.037684,0.065395,0.588644,...,0.073761,0.816040,0.076892,0.167379,0.051071,0.241700,0.025137,0.032319,0.043402,62.0
GSM491941,0.819867,0.853270,0.061275,0.137825,0.079826,0.037950,0.960325,0.034399,0.076923,0.582535,...,0.070806,0.436428,0.081855,0.163969,0.046700,0.147942,0.026827,0.040882,0.041331,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM492472,0.850640,0.811812,0.046001,0.165387,0.074648,0.031145,0.952253,0.036039,0.093094,0.636976,...,0.066265,0.830900,0.090231,0.142626,0.060186,0.371981,0.030636,0.045670,0.048369,58.0
GSM492473,0.785949,0.863659,0.065620,0.149490,0.078410,0.028611,0.958846,0.035945,0.081228,0.607439,...,0.059076,0.827640,0.080969,0.172553,0.044991,0.433100,0.034338,0.033494,0.037386,61.0
GSM492474,0.827493,0.815332,0.055990,0.155754,0.091335,0.042109,0.954158,0.040996,0.084361,0.607488,...,0.086786,0.828041,0.092011,0.162151,0.056461,0.422606,0.035688,0.035118,0.043518,72.0
GSM492475,0.817919,0.860943,0.062512,0.129134,0.065681,0.025430,0.966155,0.045333,0.075592,0.611718,...,0.090769,0.839257,0.098245,0.169964,0.047952,0.433005,0.034179,0.040766,0.048207,65.0


In [38]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV

# Assuming X_genomic is your genomic dataset and y_age is the corresponding age labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the ElasticNetCV model
alpha_values = np.logspace(-3, 3, 10)  # Range of alpha values to search
l1_ratio_values = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]  # Range of l1_ratio values to search
elasticnet_cv = ElasticNetCV(alphas=alpha_values, l1_ratio=l1_ratio_values, cv=5)
elasticnet_cv.fit(X_train, y_train)

# Print the selected alpha and l1_ratio values
print("Selected alpha:", elasticnet_cv.alpha_)
print("Selected l1_ratio:", elasticnet_cv.l1_ratio_)

# Evaluate the model on the test set
test_score = elasticnet_cv.score(X_test, y_test)
print("Test Set R^2 Score:", test_score)


e:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.152e+00, tolerance: 2.345e+00
  model = cd_fast.enet_coordinate_descent(
e:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.099e+02, tolerance: 2.345e+00
  model = cd_fast.enet_coordinate_descent(
e:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.478e+02, tolerance: 2.345e+00
  model = cd_fast.enet_coo

Selected alpha: 0.004641588833612777
Selected l1_ratio: 0.1
Test Set R^2 Score: 0.3855392821019268


e:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.207e+02, tolerance: 2.881e+00
  model = cd_fast.enet_coordinate_descent(


In [45]:
#####  feature selection
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
X_feat = X
y_feat = y
# X_feat, y = make_regression(n_samples=100, n_features=100, noise=0.1, random_state=42)
model = ElasticNet(alpha=0.00464, l1_ratio=0.1) 
model.fit(X_feat, y_feat)
feature_importance = model.coef_
sorted_indices = np.argsort(np.abs(feature_importance))[::-1]
top_features = sorted_indices[:100]
# print("Top features:")
# for idx in top_features:
#     print(f"Feature {idx}: {feature_importance[idx]}")
top_feature_names = X_feat.columns[top_features]
top_feature_coefficients = feature_importance[top_features]

# Create a DataFrame with feature names and their coefficients
feature_data = pd.DataFrame({
    'Feature': top_feature_names,
    'Coefficient': top_feature_coefficients
})

print("Top features and their coefficients:")
print(feature_data)

# Output to CSV
feature_data.to_csv('top_features.csv', index=False)

Top features and their coefficients:
       Feature  Coefficient
0   cg02228185    -2.801396
1   cg22580512    -2.403753
2   cg02288165     2.280634
3   cg22549408     2.252155
4   cg05740244    -2.014717
..         ...          ...
95  cg16986846     1.073531
96  cg15364618    -1.072202
97  cg02082342    -1.071462
98  cg08888956    -1.070986
99  cg20812929    -1.066942

[100 rows x 2 columns]


e:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.981e+02, tolerance: 3.586e+00
  model = cd_fast.enet_coordinate_descent(


In [58]:
choose = cb[['age', 'cg02228185',	'cg22580512',	'cg02288165',	'cg22549408',	'cg05740244',	'cg00019495',	'cg23762517',	'cg10935723',	'cg07327347',	'cg18771300',	'cg07884019',	'cg10523019',	'cg25133685',	'cg06590711',	'cg00141845',	'cg02654291',	'cg07388493',	'cg07403350',	'cg24429836',	'cg23124451',	'cg25004981',	'cg03454353',	'cg14088161',	'cg14826456',	'cg22736354',	'cg16474696',	'cg09835085',	'cg01512854',	'cg12650635',	'cg17901463',	'cg11126134',	'cg17471102',	'cg03923277',	'cg26987645',	'cg06771126',	'cg22333868',	'cg07693270',	'cg04019407',	'cg13460409',	'cg10203483',	'cg20797216',	'cg25681177',	'cg16685388',	'cg19258882',	'cg22983460',	'cg20916523',	'cg01136458',	'cg13269407',	'cg17073891',	'cg13301003',	'cg14951292',	'cg20945531',	'cg04230060',	'cg16005224',	'cg13944141',	'cg22901840',	'cg13899108',	'cg13119609',	'cg16440909',	'cg06906435',	'cg17655614',	'cg05254747',	'cg19856594',	'cg03330058',	'cg04123409',	'cg04037228',	'cg03764585',	'cg23504246',	'cg13749822',	'cg01693350',	'cg07905963',	'cg23894058',	'cg21053323',	'cg16744741',	'cg23771661',	'cg06791102',	'cg23180489',	'cg21120249',	'cg11108890',	'cg16742703',	'cg00451635',	'cg04384208',	'cg13033054',	'cg01837574',	'cg25809905',	'cg19421044',	'cg25345738',	'cg25569462',	'cg20977864',	'cg04300115',	'cg06143901',	'cg09554443',	'cg17704839',	'cg10978346',	'cg12215675',	'cg16986846',	'cg15364618',	'cg02082342',	'cg08888956',	'cg20812929']]

In [80]:
###### XGBoost

import xgboost as xgb

#X_choose, y = make_regression(n_samples=100, n_features=100, noise=0.1, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(choose.drop('age', axis=1), choose['age'], test_size=0.2, random_state=42)

# Train an XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.5, max_depth=2, reg_alpha=1.05, reg_lambda=0.1)
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='rmse', early_stopping_rounds=10, verbose=False)

# Evaluate the model
train_score = xgb_model.score(X_train, y_train)
test_score = xgb_model.score(X_test, y_test)
print("XGBoost model train score:", train_score)
print("XGBoost model test score:", test_score)

XGBoost model train score: 0.866776941054914
XGBoost model test score: 0.4983150409058519


e:\Anaconda\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
e:\Anaconda\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [81]:
from sklearn.metrics import mean_squared_error

# Predictions on the training and testing sets
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Calculate the Mean Squared Error
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on training set:", mse_train)
print("MSE on testing set:", mse_test)

from sklearn.metrics import r2_score, roc_auc_score

# Calculate R-squared (R2) score
r2 = r2_score(y_test, y_test_pred)
print("R-squared (R2) score:", r2)


MSE on training set: 8.883535351900989
MSE on testing set: 32.74625557885628
R-squared (R2) score: 0.4983150409058519


In [82]:
#########random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20],      # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]     # Minimum number of samples required to be at a leaf node
}

# Instantiate the Random Forest regressor
rf_model = RandomForestRegressor(random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print("Best parameters:", best_params)

# Evaluate the best model
y_train_pred = best_rf_model.predict(X_train)
y_test_pred = best_rf_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on training set with best model:", mse_train)
print("MSE on testing set with best model:", mse_test)


Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 150}
MSE on training set with best model: 7.270987812522735
MSE on testing set with best model: 37.20199270473101


In [92]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Instantiate a new Random Forest model with the best parameters
best_rf_model = RandomForestRegressor(n_estimators=150,
                                       #max_depth=5,
                                       min_samples_split=2,
                                       min_samples_leaf=2,
                                       random_state=42)

# Fit the model to the training data
best_rf_model.fit(X_train, y_train)

# Evaluate the model on the testing data
y_test_pred = best_rf_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)

print("MSE on testing set with best model:", mse_test)


MSE on testing set with best model: 37.20199270473101


In [93]:
####### SVM

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error

# Instantiate the SVM regressor with the desired hyperparameters
svm_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# Fit the SVM model to the training data
svm_model.fit(X_train, y_train)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'epsilon': [0.1, 0.2, 0.5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_svm_model = grid_search.best_estimator_

print("Best parameters:", best_params)

# Evaluate the best model
y_train_pred = best_svm_model.predict(X_train)
y_test_pred = best_svm_model.predict(X_test)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print("R-squared on training set with best model:", train_r2)
print("R-squared on testing set with best model:", test_r2)
print("MAE on training set with best model:", train_mae)
print("MAE on testing set with best model:", test_mae)
print("MSE on training set with best model:", train_mse)
print("MSE on testing set with best model:", test_mse)


Best parameters: {'C': 10, 'epsilon': 0.5, 'kernel': 'linear'}
R-squared on training set with best model: 0.6957196926436576
R-squared on testing set with best model: 0.4110868770709625
MAE on training set with best model: 3.2878876823905845
MAE on testing set with best model: 4.64367091547614
MSE on training set with best model: 20.289917441406033
MSE on testing set with best model: 38.4398600906782
